In [1]:
import pandas as pd
%cd ..
res = pd.read_csv('connected_models/generated_vul/juliet_vul_func1.csv')
data = pd.read_csv('Dataset_VulGen/non-vulnerable-funcs-juliet_after_delete_print.csv')

/sise/home/urizlo/Fine_tuning_T5


In [2]:
from torch import le


indexs = []
for i in range(len(res)):
    if type(res['func1_local'].iloc[i]) != float and "print" in res['func1_local'].iloc[i]:
        indexs.append(i)
print(len(indexs))

629


In [3]:
res = res.drop(indexs)
res = res.reset_index(drop=True)
data = data.drop(indexs)
data = data.reset_index(drop=True)

In [4]:
data.to_csv('Dataset_VulGen/non-vulnerable-funcs-juliet_after_delete_print.csv', index=False)

In [5]:
res.to_csv('connected_models/localization_results/func1_local_remove_print.csv', index=False)

In [12]:
import json
import random

file_path = "./Datasets/diversevul_20230702.json"

data = []
with open(file_path, "r") as file:
    for line in file:
        try:
            json_obj = json.loads(line)
            data.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON on line: {line}")
            print(f"Error details: {e}")
for d in data:
    del d['cwe']
    del d['project']
    del d['commit_id']
    del d['hash']
    del d['size']
    del d['message']

# Filter samples with target == 0
filtered_data = [d for d in data if d.get('target') == 0 and len(d.get('func')) < 4000]

# Randomly select 1000 samples
random_samples = random.sample(filtered_data, k=2000)

# Write the selected samples to a new JSON file
new_file_path = "./Datasets/non-vul_1000_samples.json"
with open(new_file_path, "w") as file:
    for sample in random_samples:
        json.dump(sample, file)
        file.write('\n')



In [2]:
import pandas as pd

vul = pd.read_csv('./generated_vul/diversevul_final.csv')
count = 0
for index, row in vul.iterrows():
    if type(row['vul']) != float and row['vul'] not in ['IS NAN', 'no location', 'wrong location'] and not row['vul'].startswith('Error') and not row['vul'].startswith('list index out of range'):
        count += 1
print(count)

1560


In [3]:
import pandas as pd

df = pd.read_csv('./generated_vul/vulgen_res.csv')
print(len(df))

989


In [2]:
import hashlib
def drop_duplicates(df):
    function_groups = {}

    for i in range(len(df)):
        nonvul = df['nonvul'].iloc[i]
        lines_after_fix = df['lines_after_fix'].iloc[i]
        
        # Split the file content into functions (assuming functions are well-defined)
        row = nonvul + lines_after_fix  # Change this based on your function definitions
        function_hash = hashlib.sha256(row.encode()).hexdigest()
        if function_hash not in function_groups:
            function_groups[function_hash] = []
        function_groups[function_hash].append(i)

    indexes_to_drop = [index for index_list in function_groups.values() if len(index_list) > 1 for index in index_list[1:]]
    df = df.drop(indexes_to_drop)
    df = df.reset_index(drop=True)
    return df

In [95]:
import pandas as pd
import pickle
nonvul = pd.read_csv('../Dataset_VulGen/vulgen_test_with_diff_lines_spaces.csv')
nonvul = nonvul.fillna("")
local = pd.read_csv('./localization_results/local_res_drop_duplicate.csv')
inject = pd.read_csv('./injection_results/inject_res_drop_duplicate.csv')
# nonvul = nonvul['nonvul'].tolist()
local = local['local_res'].tolist()
inject = inject['inject_res'].tolist()

In [96]:
for i, (loc, inj, nonvu) in enumerate(zip(local, inject, nonvul['nonvul'].tolist())):
    l = loc.split('\n')
    inj = inj.split('\n')
    non = nonvu.split('\n')
    if len(l) < len(inj) and len(non) < 20:
        print(i)
        # break

30
40
121
280
342
447
680
721
895
898
925
951
985
1002
1003
1013


In [129]:
print(local[40])

     return cdc_ncm_bind_common(dev, intf, CDC_NCM_DATA_ALTSETTING_NCM, 0); }


In [130]:
print(inject[40])

A:static int cdc_ncm_bind(struct usbnet *dev, struct usb_interface *intf) {<endRow>    int ret;
     ret = cdc_ncm_bind_common(dev, intf, CDC_NCM_DATA_ALTSETTING_NCM, 0);<s>     usbnet_link_change(dev, 0, 0);<s>     return ret; }


In [131]:
print(nonvul['vul'].iloc[40])

static int cdc_ncm_bind(struct usbnet *dev, struct usb_interface *intf) {
    int ret;
    if (cdc_ncm_select_altsetting(intf) != CDC_NCM_COMM_ALTSETTING_NCM)
        return -ENODEV;
    ret = cdc_ncm_bind_common(dev, intf, CDC_NCM_DATA_ALTSETTING_NCM, 0);
    usbnet_link_change(dev, 0, 0);
    return ret; }


In [98]:
print(nonvul['nonvul'].iloc[121])

int cosine_seek_read(wtap *wth, gint64 seek_off, struct wtap_pkthdr *phdr, Buffer *buf, int *err, gchar **err_info) {
    char line[COSINE_LINE_LENGTH];
    if (file_seek(wth->random_fh, seek_off, SEEK_SET, err) == -1)
        return FALSE;
    if (file_gets(line, COSINE_LINE_LENGTH, wth->random_fh) == NULL) {
        *err = file_error(wth->random_fh, err_info);
        if (*err == 0) {
            *err = WTAP_ERR_SHORT_READ; }
        return FALSE; }
    return parse_cosine_packet(wth->random_fh, phdr, buf, line, err, err_info); }


In [45]:
def find_non_continuous_indices(nonvul, local):
    non_continuous_indices = []

    for i, (function_code, local_code) in enumerate(zip(nonvul, local)):
        # Split the function and local code into lines
        function_lines = function_code.split('\n')
        local_lines = local_code.split('\n')
        in_row = False
        start = False
        loc = local_lines[0]
        l1 = 0
        for l2, line in enumerate(function_lines):
            if loc.strip() == line.strip():
                while local_lines[l1].strip() == function_lines[l2].strip():
                    start = True
                    l1 += 1
                    l2 += 1
                    if l1 >= len(local_lines) or l2 >= len(function_lines):
                        in_row = True
                        break
                if not in_row:
                    non_continuous_indices.append(i)
            if start:
                break
            

    return non_continuous_indices

non_continuous_indices = find_non_continuous_indices(nonvul, local)
print(len(non_continuous_indices))

77


In [46]:
non_continuous_indices

[32,
 36,
 41,
 56,
 69,
 71,
 101,
 104,
 109,
 111,
 116,
 123,
 130,
 156,
 167,
 212,
 220,
 229,
 237,
 238,
 239,
 242,
 246,
 255,
 260,
 279,
 311,
 324,
 358,
 360,
 361,
 364,
 368,
 384,
 401,
 419,
 434,
 436,
 493,
 494,
 498,
 510,
 528,
 568,
 616,
 618,
 632,
 637,
 638,
 639,
 647,
 650,
 669,
 698,
 704,
 709,
 755,
 763,
 815,
 820,
 824,
 861,
 907,
 912,
 950,
 952,
 966,
 992,
 1007,
 1014,
 1046,
 1047,
 1048,
 1049,
 1051,
 1059,
 1062]

In [75]:
print(local[5])

     if (ps_dec->u4_init_dec_flag && ps_dec->s_prev_seq_params.u1_eoseq_pending) {
         if ((ps_cur_slice->u1_slice_type!= I_SLICE) && (ps_dec->i4_degrade_pic_cnt!= ps_dec->i4_nondegrade_interval))
             ps_cur_slice_type = PIC_TYPE_I;
